# Mass Spring Damper System
### mx'' + cx' + kx = u
###### https://en.wikipedia.org/wiki/Mass-spring-damper_model

![Image of mass-spring-damper system](http://www.roboticslab.ca/wp-content/uploads/2012/11/msd.png)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from matplotlib.pyplot import rcParams
from utils import plot_xv

### We start with the most basic thing: equations of motion and integrating trajectories (u=0)
##### We use https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html

In [2]:
from scipy.integrate import solve_ivp

In [3]:
# This is the implementation of the mass-spring-damper that needs to be integrated, 
# using solve_ivp (initial value problem)
def msd_int(t, y, m=1, c=1, k=1):
    v,x = y
    dxdt = v
    dvdt = - v*c/m - x*k/m
    return [dvdt, dxdt]

In [4]:
rcParams['figure.figsize'] = 12, 4
@interact(m=(1,10,1), c=(0,10), k=(0,10))
def plot_interactive_msd(m,c,k):
    y = solve_ivp(msd_int, y0=[0,1], t_span=(0,10), t_eval=np.arange(0,10,0.01), args=(m,c,k))
    plot_xv(y.y[1], y.y[0], y.t, title='Unforced Dynamics', 
            ax1ylim=[-0.6,1.1], ax2xlim=[-0.5,1.1], ax2ylim=[-0.8,0.2])

interactive(children=(IntSlider(value=5, description='m', max=10, min=1), IntSlider(value=5, description='c', …

### And then add inputs
##### We use https://python-control.readthedocs.io/en/0.8.3/index.html#

In [5]:
import control as ct

In [6]:
# This function builds the mass-spring-damper state-space (msd_ss) object
def get_msd_ss(m=1,c=1,k=1, fullstate=False):
    A = np.array([[-c/m, -k/m],
                  [1, 0]])
    B = np.array([[1],
                  [0]])
    if fullstate:
        C = np.eye(2)
    else:
        C = np.array([0,1])
    D = 0
    msd_ss = ct.ss(A,B,C,D)
    return msd_ss

In [42]:
# Note that here we get the exact same plots as above, except using the control library instead
# But instead of initial_response we could also use step_response, impulse_response, etc.
@interact(m=(1,10,1), c=(0,10), k=(0,10))
def plot_interactive_msd_step(m,c,k):
    msd_ss = get_msd_ss(m,c,k)
    t,y,z = ct.initial_response(msd_ss, X0=[0,1], return_x=True)
    plot_xv(z[1], z[0], t, title='Step Response')

interactive(children=(IntSlider(value=5, description='m', max=10, min=1), IntSlider(value=5, description='c', …

### We finally actually control the system, first with PID

In [87]:
m=1; c=1; k=1;
msd_ss = get_msd_ss(m,c,k)
# We transform the state-space representation into transfer-function representation, because this makes it easier
# to cascade with PI controllers, which have simple tf representation (kp + ki/s)
msd_tf = ct.ss2tf(msd_ss)
print(msd_tf)


     1
-----------
s^2 + s + 1



In [50]:
# Here we build an approximate PID controller
# Note "approximate" because PID with "real" derivative term (kd*s) don't have a tf realization!
# See http://www.cds.caltech.edu/~murray/books/AM08/pdf/fbs-pid_17Aug2019.pdf for details
def get_pid_tf(kp=1, ki=1, kd=1, tau=1):
    s = ct.tf('s')
    pid_tf = kp + ki/s + kd*s/(tau*s+1)
    return pid_tf

In [103]:
# Then we get the closed loop tf
#  r--+-->[C]-->[P]-->y
#     |               |
#     ^----[-1]<------|
@interact(kp=(0,10), ki=(0,10), kd=(0,10))
def plot_interactive_msd_step(kp=1,ki=1,kd=0):
    pid_tf = get_pid_tf(kp,ki,kd)
    closed_tf = ct.feedback(pid_tf*msd_tf,1)
    print("closed tf", closed_tf)
    t,y=ct.step_response(closed_tf)
    plt.plot(y)
    plt.ylim([-0.1,1.6])

interactive(children=(IntSlider(value=1, description='kp', max=10), IntSlider(value=1, description='ki', max=1…

### Now LQR

In [163]:
m=1; c=1; k=1;
msd_ss = get_msd_ss(m,c,k, fullstate=True)
A = msd_ss.A
B = msd_ss.B

In [166]:
Klqr,_,_ = ct.lqr(A,B,np.eye(2),1)

Not sure how to deal with change of variables (if I want LQR to not lead to (0,0) state)... TODO!

# Bibliography
http://ctms.engin.umich.edu/CTMS/index.php?example=Introduction&section=SystemModeling